<a href="https://colab.research.google.com/github/zzwony/Start_0920/blob/main/01_11_bert_ner_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 의존성 패키지 설치

In [ ]:
!pip install ratsnlp

In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 인퍼런스 설정

In [7]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들인다.

In [9]:
import torch
from transformers import BertConfig, BertConfig, BertForTokenClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

# 레이블 맵 작성
범주 인덱르를 범주명과 매칭하는 사전을 만든다.
- 문자: (선택 사항) 주어진 문자는 원래 문자열의 시작 또는 끝에서 제거된다.
- 문자 매개변수가 제공되지 않으면 문자열의 시작과 끝에서 공백이 제거된다.

In [10]:
## strip()은 공백제거 역할을 함.
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

# 인퍼런스 함수 선언
인퍼런스 함수를 선언한다.

In [11]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)## 예측 값에서 top-k의 결과를 받고 싶을 때
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

In [13]:
sentence = '대한민국 인구수는 5천만명이다.'
inference_fn(sentence)

{'sentence': '대한민국 인구수는 5천만명이다.',
 'result': [{'token': '대한민국', 'predicted_tag': '기관명', 'top_prob': '0.7882'},
  {'token': '인구', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '##수는', 'predicted_tag': 'O', 'top_prob': '0.9952'},
  {'token': '5천만', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9819'},
  {'token': '##명이', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9898'},
  {'token': '##다', 'predicted_tag': 'O', 'top_prob': '0.9972'},
  {'token': '.', 'predicted_tag': 'O', 'top_prob': '0.9997'}]}

In [14]:
sentence = '대한민국 인구수는 5천만명이다.'
inference_fn(sentence)

{'sentence': '대한민국 인구수는 5천만명이다.',
 'result': [{'token': '대한민국', 'predicted_tag': '기관명', 'top_prob': '0.7882'},
  {'token': '인구', 'predicted_tag': 'O', 'top_prob': '0.9993'},
  {'token': '##수는', 'predicted_tag': 'O', 'top_prob': '0.9952'},
  {'token': '5천만', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9819'},
  {'token': '##명이', 'predicted_tag': '기타 수량표현', 'top_prob': '0.9898'},
  {'token': '##다', 'predicted_tag': 'O', 'top_prob': '0.9972'},
  {'token': '.', 'predicted_tag': 'O', 'top_prob': '0.9997'}]}

# 웹서비스 만들기 준비

In [17]:
!mkdir /root/.ngrok2 && echo 'authtoken: 2K7I7O1DpP2jRhLZkbgOyok0X8H_6ZVLBrT25o6NA81BJMC9a' > /root/.ngrok2/ngrok.yml

In [18]:
from ratsnlp.nlpbook.paircls import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.paircls.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2c20-34-90-34-111.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 03:22:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2023 03:22:00] "GET /favicon.ico HTTP/1.1" 404 -
